In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt     
import os 
from glob import glob
import json


In [8]:
class Preprocesser(object):

    def __init__(self,files:list):
        self.dices_raw = self.get_all_dices(files)
        self.dices = self.convert_to_six(self.dices_raw)
        
    def get_training_data(self,file_array: list):
        training_data = []
   
        #looping through files array
        for file in file_array:
            #getting the image data
            image = tf.keras.utils.load_img(file, grayscale=False, color_mode='grayscale', target_size=None,interpolation='nearest')
            #converting everything to numpy array
            input_arr = tf.keras.preprocessing.image.img_to_array(image)
            input_arr = np.array([input_arr]) 
            #normalize the array to a float from 0 to 1
            input_arr = input_arr/255
            #add dice image to training data
            training_data.append(input_arr[0])

        dices_array = np.array(training_data)
        return dices_array

    
    def get_all_dices(self,files: list):
        dices = []
        for file in files:
            f = glob(file)
            dices.append(self.get_training_data(f))
        return dices

    def convert_to_six(self, dices_raw:list):
        dices = []
        for i in range(7):
            if i == 1:
                temp = np.concatenate([dices_raw[i], dices_raw[i+1]])
                dices.append(temp)
                continue
            if i ==2:
                continue
            if i == 3:
                temp = np.concatenate([dices_raw[i], dices_raw[i+1]])
                dices.append(temp)
                continue
            if i ==4:
                continue
            else:
                dices.append(dices_raw[i])

        t = np.concatenate([dices_raw[7],dices_raw[8]])

        t= np.concatenate([t,dices_raw[9]])
        t= np.concatenate([t,dices_raw[10]])
        dices.append(t)
        return dices

    def normalize_samples(max_samples: int, dices: list):
        #normalizing the amount of dices
        _MAX_SAMPLES = max_samples
        _dices = []
        for dice in dices:
            _dices.append(dice[:_MAX_SAMPLES])

        _dices = np.array(_dices)

        return _dices
    
    


In [14]:
class Training_data(object):

    files = ['./train_set/00/*','./train_set/01/*','./train_set/02/*','./train_set/03/*','./train_set/04/*','./train_set/05/*','./train_set/06/*','./train_set/07/*','./train_set/08/*','./train_set/09/*','./train_set/10/*'] 
    def __init__(self):
        preprocessing = Preprocesser(self.files)
        self.dices = preprocessing.dices
        self.labels = self.get_labels(self.dices) 
        self.dices = self.stack_training_data(self.dices)
        self.save_data(self.dices,'training_data.npy')
        self.save_data(self.labels,'labels.npy')
    

    def save_data(self, data, name):

        np.save(name,data)
    #getting all the labels for this training
    def get_labels(self,_dices):
        labels = []
        for i in range(len(_dices)):
            for j in range(len(_dices[i])):
                labels.append(i)
        arr = np.array(labels)
        arr = arr.reshape((-1,1))
        return arr

    def stack_training_data(self,_dices: list):
        training_data = np.concatenate([_dices[0],_dices[1]])
        for i in range(2,len(_dices)):
            training_data = np.concatenate([training_data,_dices[i]])
        return training_data

In [15]:
td = Training_data()
    